-----

KE-T5-Small Hugging-Face

-----

# Dataset Load and Slicing

In [ ]:
# 드라이브 마운팅. 이미 했으면 안 해도 됨.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


pip install

In [ ]:
!pip install transformers datasets sentencepiece evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00


In [ ]:
! pip install -U accelerate
! pip install -U transformers

# 이거 실행 후 두 줄 모두 주석 처리한 뒤, 세션 다시 시작한 후 처음부터 돌려야 함 <- 이렇게 이제 안 해도 됨

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

Get Dataset

In [ ]:
import pandas as pd

# 파일 경로를 수정하세요. 예: '/content/drive/My Drive/data/my_data.tsv'
file_path = '/content/drive/My Drive/Colab/T5-jeju/data/'
train_tsv_name = 'train.tsv'  # train data 바꾼 다음 이 이름으로 저장해두셈
test_tsv_name = 'test.tsv'  # test data 바꾼 다음 이 이름으로 저장해두셈

# TSV 파일을 로드
train_df = pd.read_csv(file_path + train_tsv_name, delimiter='\t', on_bad_lines='skip')
test_df = pd.read_csv(file_path + test_tsv_name, delimiter='\t', on_bad_lines='skip')

# 데이터 확인
train_df.head()
# test_df.head()

len(train_df)

529988

Slice dataset and load

In [ ]:
from datasets import load_dataset

# 각 데이터 셋의 샘플수를 정한다.
num_train = 480000 # 1000000
# num_valid = len(train_df) - num_train - 1 # 90000
num_valid = 45000
num_test = 8000 # 10000

print(len(train_df), len(test_df))

if num_train + num_valid > len(train_df) or num_test > len(test_df):
    raise IndexError("slicing 범위를 초과하였습니다.")

# Slice Dataset
sliced_train_df = train_df.iloc[:num_train]
sliced_valid_df = train_df.iloc[num_train:num_train+num_valid]
sliced_test_df = test_df.iloc[:num_test]

# To tsv
sliced_train_df.to_csv("sliced_train.tsv", sep='\t', index=False)
sliced_valid_df.to_csv("sliced_valid.tsv", sep='\t', index=False)
sliced_test_df.to_csv("sliced_test.tsv", sep='\t', index=False)

# Load Dataset
data_files = {"train": "sliced_train.tsv", "valid": "sliced_valid.tsv", "test": "sliced_test.tsv"}
dataset =  load_dataset("csv", data_files=data_files, delimiter="\t")

# Access Method
print(dataset['train']['standard_form'][:3], dataset['train']['dialect_form'][:3])
print(dataset['train'][:3]['standard_form'], dataset['train'][:3]['dialect_form'])

529988 8261


Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

['요세 전기세 둘이 살면 얼마 나와?', '우리 &name1& 하이', '야 시작하게 동화책 갖고 왔어?'] ['요세 전기세 둘이 살면 얼마 나와?', '우리 &name1& 하이', '야 시작하게 동화책 갖고 완?']
['요세 전기세 둘이 살면 얼마 나와?', '우리 &name1& 하이', '야 시작하게 동화책 갖고 왔어?'] ['요세 전기세 둘이 살면 얼마 나와?', '우리 &name1& 하이', '야 시작하게 동화책 갖고 완?']


# Fine-Tuning

Import

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_metric
import numpy as np
import torch
import multiprocessing

Set checkpoint

In [ ]:
model_ckpt = "KETI-AIR/ke-t5-small"
max_token_length = 64

## Tokenizer

Get tokenizer from pretraiend model checkpoint

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/597 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Example

In [ ]:
# Access Example
dataset['train'][10]['standard_form'], dataset['train'][10]['dialect_form']

('그거 웬만한 사람은 못하지', '그거 웬만한 사람은 못하주게')

In [ ]:
# Usage Example
tokenized_sample_dialect = tokenizer(dataset['train'][10]['dialect_form'],
                                max_length=max_token_length,
                                padding=True, truncation=True)
tokenized_sample_dialect

{'input_ids': [9980, 38305, 1117, 663, 504, 158, 224, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
# Usage Example
tokenized_sample_standard = tokenizer(dataset['train'][10]['standard_form'],
                                max_length=max_token_length,
                                padding=True, truncation=True)
tokenized_sample_standard

{'input_ids': [9980, 38305, 1117, 663, 218, 1], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [ ]:
# Usage Example
pd.DataFrame(
    [
        tokenized_sample_dialect['input_ids'],
        tokenizer.convert_ids_to_tokens(tokenized_sample_dialect['input_ids'])
    ], index=('ids', 'tokens')
)

,0,1,2,3,4,5,6,7
ids,9980,38305,1117,663,504,158,224,1
tokens,▁그거,▁웬만한,▁사람은,▁못,하,주,게,</s>


In [ ]:
# Usage Example
pd.DataFrame(
    [
        tokenized_sample_standard['input_ids'],
        tokenizer.convert_ids_to_tokens(tokenized_sample_standard['input_ids'])
    ], index=('ids', 'tokens')
)

,0,1,2,3,4,5
ids,9980,38305,1117,663,218,1
tokens,▁그거,▁웬만한,▁사람은,▁못,하지,</s>


Convert examples to features

In [ ]:
def convert_examples_to_features(examples):
    ###########################################################################
    # with 쓰는 옛날 방식
    # input_encodings = tokenizer(examples['dialect_form'],
    #                             max_length=max_token_length, truncation=True)

    # Setup the tokenizer for targets
    # with tokenizer.as_target_tokenizer():
    # target_encodings = tokenizer(text_target=examples['standard_form'],
    #                             max_length=max_token_length, truncation=True)
    #
    #
    # return {
    #     "input_ids": input_encodings["input_ids"],
    #     "attention_mask": input_encodings["attention_mask"],
    #     "labels": target_encodings["input_ids"]
    # }

    # 그런데 이렇게 하면 인풋하고 한번에 처리 가능함.

    model_inputs = tokenizer(examples['dialect_form'],
                             text_target=examples['standard_form'],
                             max_length=64, truncation=True)

    return model_inputs

In [ ]:
NUM_CPU = multiprocessing.cpu_count()
NUM_CPU

2

In [ ]:
tokenized_datasets = dataset.map(convert_examples_to_features,
                                 batched=True,
                                 # 이걸 쓰지 않으면 원 데이터 'en', 'ko'가 남아서
                                 # 아래서 콜레이터가 패딩을 못해서 에러남
                                 remove_columns=dataset["train"].column_names,
                                 num_proc=NUM_CPU)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/480000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/45000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/8000 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets
tokenized_datasets['train'][10]

{'input_ids': [9980, 38305, 1117, 663, 504, 158, 224, 1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [9980, 38305, 1117, 663, 218, 1]}

In [ ]:
print( '원 데이터    :', dataset['train'][10]['dialect_form'] )
print( '처리 후 데이터:', tokenized_datasets['train'][10]['input_ids'] )
print( '토큰화       :', tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][10]['input_ids']) )

print('\n')
print( '원 데이터    :', dataset['train'][10]['standard_form'] )
print( '처리 후 데이터:', tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][10]['labels']) )
print( '토큰화       :', tokenized_datasets['train'][10]['labels'] )

원 데이터    : 그거 웬만한 사람은 못하주게
처리 후 데이터: [9980, 38305, 1117, 663, 504, 158, 224, 1]
토큰화       : ['▁그거', '▁웬만한', '▁사람은', '▁못', '하', '주', '게', '</s>']


원 데이터    : 그거 웬만한 사람은 못하지
처리 후 데이터: ['▁그거', '▁웬만한', '▁사람은', '▁못', '하지', '</s>']
토큰화       : [9980, 38305, 1117, 663, 218, 1]


## Model

Call model from checkpoint

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

cuda


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

Forward Test

In [ ]:
encoder_inputs = tokenizer(
    ["게난 음식 잘하는 사람들."],
    return_tensors="pt"
)['input_ids'].to(device)

decoder_targets = tokenizer(
    ["그니까 음식 잘하는 사람들."],
    return_tensors="pt"
)['input_ids'].to(device)

print( encoder_inputs )
print( decoder_targets )

tensor([[  274,  1291,  3430, 18160,  3340,     3,     1]], device='cuda:0')
tensor([[14251,  3430, 18160,  3340,     3,     1]], device='cuda:0')


In [ ]:
decoder_inputs = model._shift_right(decoder_targets)

pd.DataFrame(
    [
        tokenizer.convert_ids_to_tokens(decoder_targets[0]),
        tokenizer.convert_ids_to_tokens(decoder_inputs[0])
    ],
    index=('decoder target', 'decoder input')
)

,0,1,2,3,4,5
decoder target,▁그니까,▁음식,▁잘하는,▁사람들,.,</s>
decoder input,<pad>,▁그니까,▁음식,▁잘하는,▁사람들,.


In [ ]:
# forward pass
outputs = model(input_ids=encoder_inputs,
                decoder_input_ids=decoder_inputs,
                labels=decoder_targets)

outputs.loss

tensor(112.4671, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
outputs['encoder_last_hidden_state'].shape

torch.Size([1, 7, 512])

In [ ]:
outputs['logits'].shape

torch.Size([1, 6, 64128])

In [ ]:
tokenizer.convert_ids_to_tokens( torch.argmax(outputs['logits'][0], axis=1).cpu().numpy() )

['原', '原', '原', '原', 'hankookilbo', 'hankookilbo']

## Collator

Create collator compatible with previous tokenizer and model

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Example

In [ ]:
# 각 항목아래 샘플들이 리스트 형태로 묶여 반환된다.
tokenized_datasets["train"][1:3]

{'input_ids': [[306, 467, 28887, 240, 413, 5134, 1],
  [2465, 41315, 11388, 3239, 903, 5691, 85, 1]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]],
 'labels': [[306, 467, 28887, 240, 413, 5134, 1],
  [2465, 41315, 11388, 3239, 903, 56047, 85, 1]]}

In [ ]:
# 콜레이터에는 샘플을 개별 {}로 넘겨야 됨
[tokenized_datasets["train"][i] for i in range(1, 3)]

[{'input_ids': [306, 467, 28887, 240, 413, 5134, 1],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1],
  'labels': [306, 467, 28887, 240, 413, 5134, 1]},
 {'input_ids': [2465, 41315, 11388, 3239, 903, 5691, 85, 1],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1],
  'labels': [2465, 41315, 11388, 3239, 903, 56047, 85, 1]}]

Padding and Shifting with collator

In [ ]:
# 콜레이터를 돌리면 알아서 패딩하고 쉬프트 시킨다.
batch = data_collator(
    [tokenized_datasets["train"][i] for i in range(1, 3)]
)

In [ ]:
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [ ]:
batch

{'input_ids': tensor([[  306,   467, 28887,   240,   413,  5134,     1,     0],
        [ 2465, 41315, 11388,  3239,   903,  5691,    85,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[  306,   467, 28887,   240,   413,  5134,     1,  -100],
        [ 2465, 41315, 11388,  3239,   903, 56047,    85,     1]]), 'decoder_input_ids': tensor([[    0,   306,   467, 28887,   240,   413,  5134,     1],
        [    0,  2465, 41315, 11388,  3239,   903, 56047,    85]])}

## Metric : BLEU

Import blue score as metric

In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

In [ ]:
# 걍 evaluate 테스트라 실행 안 해도 됨
# predictions = [
#     "저는 딥러닝을 좋아해요.",
#     "요즘은 딥러닝 프레임워크가 잘 발달되어 있기 때문에 누구의 도움 없이도 기계 번역 시스템을 구축할 수 있습니다."
# ]

# references = [
#     ["저는 딥러닝을 좋아해요.", "나는 딥러닝을 사랑해요."],
#     ["요즘은 딥러닝 프레임워크가 잘 발달되어 있기 때문에 누구의 도움 없이도 기계 번역 시스템을 구축할 수 있습니다.",
#      "최근에는 딥러닝 프레임워크가 잘 개발되어 있기 때문에 다른 사람의 도움 없이도 기계 번역 시스템을 개발할 수 있습니다."]
# ]
# metric.compute(predictions=predictions, references=references)

In [ ]:
# predictions = [
#     "저는 딥러닝을 좋아해요.",
#     "딥러닝 프레임워크가 잘 개발되었기 때문에 요즘은 누군가의 도움 없이 기계번역 시스템을 구축할 수 있다."
# ]

# references = [
#     ["저는 딥러닝을 좋아해요.", "나는 딥러닝을 사랑해요."],
#     ["요즘은 딥러닝 프레임워크가 잘 발달되어 있기 때문에 누구의 도움 없이도 기계 번역 시스템을 구축할 수 있습니다.",
#      "최근에는 딥러닝 프레임워크가 잘 개발되어 있기 때문에 다른 사람의 도움 없이도 기계 번역 시스템을 개발할 수 있습니다."]
# ]
# metric.compute(predictions=predictions, references=references)

Metric Function

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    return result

## Trainer

Set training arguments

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="chkpt",
    learning_rate=0.0005,
    weight_decay=0.01,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    num_train_epochs=1,
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy="epoch",
    logging_strategy="no",
    predict_with_generate=True,
    fp16=False,
    gradient_accumulation_steps=2,
    report_to="none" # Wandb 로그 끄기
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Set Trainer

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Train

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu
1,No log,0.389265,68.892285


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=3750, training_loss=1.4831015625, metrics={'train_runtime': 4932.1708, 'train_samples_per_second': 97.32, 'train_steps_per_second': 0.76, 'total_flos': 7391147172691968.0, 'train_loss': 1.4831015625, 'epoch': 1.0})

Save trained model

In [ ]:
trainer.save_model("./results")

## Test Saved Model

In [ ]:
model_dir = "/content/drive/MyDrive/Colab/T5-jeju/data/saved_models/test_rlhf"  # 불러올 모델 경로 설정
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

model.cpu();

Some weights of the model checkpoint at /content/drive/MyDrive/Colab/T5-jeju/data/saved_models/test_rlhf were not used when initializing T5ForConditionalGeneration: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Test with sample sentences

In [ ]:
# These are just for simple test
input_text = [
    "잘 말려 줍서",
    "보릴 무껑 눌었당 태작을 헌다"
]

In [ ]:
# These are just for simple test
inputs = tokenizer(input_text, return_tensors="pt",
                   padding=True, max_length=max_token_length)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [ ]:
# These are just for simple test
inputs

{'input_ids': tensor([[  242, 40867,     7, 33291,   125,     1,     0,     0,     0,     0,
             0,     0,     0],
        [ 1085,  2346,   873, 63935,     7, 31776,  9854,   422,  3346, 27692,
         12739,    66,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
# These are just for simple test
koreans = model.generate(
    **inputs,
    max_length=max_token_length,
    num_beams=5,
)

koreans.shape

torch.Size([2, 12])

In [ ]:
# These are just for simple test
[
    tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(korean)) for korean in koreans
]

['<pad> 잘 말려 주십시오</s><pad><pad><pad><pad><pad><pad><pad>',
 '<pad> 보리를 묶어서 가렸다가 타작을 한다</s>']

In [ ]:
from torch.utils.data import DataLoader

test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=50, collate_fn=data_collator
)

In [ ]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 480000
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8000
    })
})


### Test with test dataset

In [ ]:
iter_count = 20  # iter 돌릴 횟수
total_score = 0

test_dataloader_iter = iter(test_dataloader)

for i in range(iter_count):
  test_batch = next(test_dataloader_iter)
  test_input = { key: test_batch[key] for key in ('input_ids', 'attention_mask') }

  # sentences that model predicted(dialect form -> standard form)
  standard_sentences = model.generate(
      **test_input,
      max_length=max_token_length,
      num_beams=5,
  )

  labels = np.where(test_batch.labels != -100, test_batch.labels, tokenizer.pad_token_id)
  originals = tokenizer.batch_decode(test_batch.input_ids, skip_special_tokens=True)
  references = tokenizer.batch_decode(labels, skip_special_tokens=True)
  predictions = tokenizer.batch_decode(standard_sentences, skip_special_tokens=True )

  # evaluation of one mini batch
  cur_score = metric.compute(predictions=predictions, references=references)['score']
  total_score += cur_score
  print('iter {} score: {}'.format(i+1, cur_score))

print('total average score:', total_score / iter_count)


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
###data -> tsv

from torch.utils.data import DataLoader

test_dataloader = DataLoader(
    tokenized_datasets["train"], batch_size=10, collate_fn=data_collator
)

iter_count = 5000  # iter 돌릴 횟수
total_score = 0

test_dataloader_iter = iter(test_dataloader)

data_list = []

for i in range(iter_count):
  test_batch = next(test_dataloader_iter)
  test_input = { key: test_batch[key] for key in ('input_ids', 'attention_mask') }

  # sentences that model predicted(dialect form -> standard form)
  standard_sentences = model.generate(
      **test_input,
      max_length=max_token_length,
      num_beams=5,
  )

  labels = np.where(test_batch.labels != -100, test_batch.labels, tokenizer.pad_token_id)
  originals = tokenizer.batch_decode(test_batch.input_ids, skip_special_tokens=True)
  references = tokenizer.batch_decode(labels, skip_special_tokens=True)
  predictions = tokenizer.batch_decode(standard_sentences, skip_special_tokens=True )

  data_list.append([originals, references, predictions])

df = pd.DataFrame(data_list, columns=['originals', 'ref', 'pred'])

df.to_csv('rm.tsv', sep='\t', index=False)


KeyboardInterrupt: 

In [ ]:
# 여기서부터는 이전 코드(iter 1회만 시행)

# test_dataloader_iter = iter(test_dataloader)
# test_batch = next(test_dataloader_iter)

In [ ]:
# test_input = { key: test_batch[key] for key in ('input_ids', 'attention_mask') }

# # sentences that model predicted(dialect form -> standard form)
# standard_sentences = model.generate(
#     **test_input,
#     max_length=max_token_length,
#     num_beams=5,
# )

In [ ]:
# labels = np.where(test_batch.labels != -100, test_batch.labels, tokenizer.pad_token_id)

In [ ]:
# originals = tokenizer.batch_decode(test_batch.input_ids, skip_special_tokens=True)

In [ ]:
# references = tokenizer.batch_decode(labels, skip_special_tokens=True)

In [ ]:
# predictions = tokenizer.batch_decode(standard_sentences, skip_special_tokens=True )

In [ ]:
# evaluation of one mini batch
# metric.compute(predictions=predictions, references=references)

In [ ]:
# for s in zip(originals, references, predictions):
#     print('Dialect   :', s[0]) # original dialect form
#     print('Reference :', s[1]) # reference standard form(answer)
#     print('Translated:', s[2]) # translated standard form(by model)
#     print('\n')

In [ ]:
from google.colab import drive, files
from requests import get
from urllib.parse import unquote

drive.mount('/mnt/')
filename = unquote(get('http://172.28.0.12:9000/api/sessions').json()[0]['name'])
filepath = f'/mnt/My Drive/Colab Notebooks/{filename}'
output_file = '/content/T5_jeju.html'

!jupyter nbconvert '{filepath}' --output '{output_file}' --to html
files.download(output_file)

Mounted at /mnt/
[NbConvertApp] Converting notebook /mnt/My Drive/Colab Notebooks/T5_jeju.ipynb to html
Traceback (most recent call last):
  File "/usr/local/bin/jupyter-nbconvert", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/jupyter_core/application.py", line 283, in launch_instance
    super().launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/nbconvert/nbconvertapp.py", line 423, in start
    self.convert_notebooks()
  File "/usr/local/lib/python3.10/dist-packages/nbconvert/nbconvertapp.py", line 597, in convert_notebooks
    self.convert_single_notebook(notebook_filename)
  File "/usr/local/lib/python3.10/dist-packages/nbconvert/nbconvertapp.py", line 560, in convert_single_notebook
    output, resources = self.export_single_notebook(
  File "/usr/local/lib/python3.10/dist-packag

FileNotFoundError: Cannot find file: /content/T5_jeju.html